In [73]:
#CFPB 11/21/2017 K. David Roell

from collections import OrderedDict
import io
import json
from math import trunc
import pandas as pd
import requests

#MSA FIPS mismatch between CFPB and FFIEC outputs, check nulls and such

In [2]:
#This code will assemble Census data that is used in the FFIEC LAR publication and A&D reports.

#Census data used will either be from the ACS 5 year 2015 data or the 2010 Decennial Summary Files
#FFIEC Census data policy requies data updates on a 5 year cycle. For HMDA 2017 the ACS5 2015 will be used
#This notebook will contain code that assembles the ACS 5 year data into a flat file that approximates the 


#ACS 5 year data fields as used by the FRB:
#Total Persons: file index 15, ACS 2015 field: B01001_1
#Min Pop Pct: file index 21, ACS 2015: if B3002_1 not 0, then (B03002_1 - B03002_3)*100 / B03002_1
#Owner Occupied Units: file index 916, ACS 2015: B25032_2
#1-4 Units: file index 900, ACS 2015: B25024_2 + B25024_3 + B25024_4 + B25024_5 + B25024_10 + B25024_11
#Tract MFI: file index 581, ACS 2015: B19113_1
#Tract to MSA pct: file index 13, ACS 2015: TBD
#Tract Median Housing Age: file index 953, ACS 2015: B25035_1 if not 0

#External links:
#Census ACS5 data field list: https://api.census.gov/data/2015/acs/acs5/variables.html
#FFIEC Census flat file published here: https://www.ffiec.gov/censusapp.htm
#FFIEC Census Data Policy: https://www.ffiec.gov/press/pr101911_ACS.htm
#Notes on HUD/FFIEC MFI for MSA/MD: HUD calculates using methodology 
#here: https://www.huduser.gov/portal/datasets/il/il2017/2017MedCalc.odn
#If OMB updated a geography and HUD did not use the update FFIEC calculates the MSA/MD MFI
#the methodology is in the word doc that is zipped with the census flat file

In [26]:
#get state code data from platform file
#These state codes will be used to iterate over API endpoings when pulling data
state_data_url = "https://raw.githubusercontent.com/cfpb/hmda-platform/master/census/src/main/resources/state.csv"
state_data = requests.get(state_data_url).content #get data from file
state_codes = pd.read_csv(io.StringIO(state_data.decode('utf-8')),sep="|", dtype=object) #convet to dataFrame
state_code_list = list(state_codes.STATE)
no_states = ['60', '66', '69', '74', '78'] #no Census API for these state codes
#America Samoa, Guam, Virgin Islands (66, 69 not found on FIPS list)
for code in no_states:
    state_code_list.remove(code)

In [27]:
#get cbsa data from platform file
#The tract to CBSA file will provide a crosswalk between Census tract level data and the appropriate MSA.
cbsa_url = "https://raw.githubusercontent.com/cfpb/hmda-platform/master/model/jvm/src/main/resources/tract_to_cbsa_2015.txt"
cbsa_data = requests.get(cbsa_url).content #get data from file
use_cols = ["state","countyFips", "geoIdMsa", "metDivFp", "tracts"] #set list of columns to use
cbsa_cols = ["name", "metDivName", "state", "countyFips", "county", "tracts","geoIdMsa", "metDivFp", "smallCounty", 
             "stateCode", "tractDecimal"]
cbsa_df = pd.read_csv(io.StringIO(cbsa_data.decode("utf-8")),sep="|", usecols=use_cols, 
                      header=None, names=cbsa_cols,dtype=object) #convert data to DataFrame
cbsa_df["fips"] = cbsa_df.apply(lambda x: x.countyFips+x.tracts, axis=1)
cbsa_df.head()

,state,countyFips,tracts,geoIdMsa,metDivFp,fips
0,06,06037,535501,31080,31084,06037535501
1,41,41005,022208,38900,NaN,41005022208
2,04,04013,610900,38060,NaN,04013610900
3,42,42017,103400,37980,33874,42017103400
4,37,37183,054403,39580,NaN,37183054403


In [63]:
#Get tract level data from Census 2015 ACS5 API

#FFIEC notes the following regarding MSA median family income calculations
#B19001 FAMILY INCOME IN THE PAST 12 MONTHS (IN 2015 INFLATION-ADJUSTED DOLLARS)
#B19113_1 at MSA level or calculated from B19101 As necessary (calculation unclear)

#ACS5 data fields
#Tract to MSA pct: file index 13, ACS 2015: B19113_1 (tract) / B19113_1 (MSA)
#median family income (MFI): B19113_1 
#total persons (population): B01001_1 
#Minority Population Percent: if B3002_1 not 0, then (B03002_1 - B03002_3)*100 / B03002_1 
#B25032_2 number of owner occupied units

#1-4 family units: B25024_2 + B25024_3 + B25024_4 + B25024_5 + B25024_10 + B25024_11 
#Median housing age: 2015 - B25035_1 if not 0: (Endpoint is for year built)

tract_acs5_fields = OrderedDict({"B19113_001E":"tract_mfi", "B01001_001E":"population", "B25032_002E":"owner_occ", 
               "B03002_003E":"min_pop","B25024_003E":"attached","B25024_002E":"detached", 
               "B25024_004E":"units_2", "B25024_005E":"units_4", "B25024_010E":"mobile",
               "B25024_011E":"van", "B25035_001E":"med_yr_built"})
#add for loop to cycle over states
first_state = True
for state_code in state_code_list:
    first = True
    tract_acs5_url = "https://api.census.gov/data/2015/acs5?get="
    for col in tract_acs5_fields.keys():
        if first:
            tract_acs5_url = tract_acs5_url + col
            first = False
        else:
            tract_acs5_url = tract_acs5_url + "," + col
    tract_acs5_url = tract_acs5_url + "&for=tract:*&in=state:"+state_code
    print(tract_acs5_url)
    state_data = requests.get(tract_acs5_url)
    if first_state:
        first_state = False
        tract_acs5_df = pd.DataFrame(json.loads(state_data.content))
        tract_acs5_df.drop([0], inplace=True)
#        print("first--------",state_data_df.tail())
    else:
        state_data_df = pd.DataFrame(json.loads(state_data.content))
        state_data_df.drop([0], inplace=True)
        tract_acs5_df = pd.concat([tract_acs5_df, state_data_df])
#        print("other--------",tract_acs5_df.tail())
    #concat each state onto first
#
 

name_cols = list(tract_acs5_fields.values()) + ["state", "county", "tract"] #set column name list
tract_acs5_df.columns = name_cols #set column names
tract_acs5_int_cols = ["tract_mfi", "owner_occ", "attached", "detached", "units_2", "units_4", "mobile", "van",
             "med_yr_built", "population", "min_pop"]
tract_acs5_df["tract_mfi"].fillna(0, inplace=True) #fill NA values to 0
tract_acs5_df["med_yr_built"].fillna(0, inplace=True) #fill NA values to 0


for col in tract_acs5_int_cols: 
    tract_acs5_df[col] = tract_acs5_df[col].map(lambda x: int(x)) #convert to ints
tract_acs5_df["total_1_to_4"] = tract_acs5_df.apply(lambda x: x.attached + x.detached + x.units_2 + x.units_4
                                       + x.mobile + x.van, axis=1)
tract_acs5_df["med_age"] = tract_acs5_df.apply(lambda x: 2015 - x.med_yr_built,axis=1) #calculate median age based on 2015
tract_acs5_df["min_pop_pct"] = tract_acs5_df.apply(lambda x: round(100 - (x.min_pop / x.population) * 100),2) if x.population !=0 else 0, axis=1)
tract_acs5_df.drop(["attached", "detached", "units_2", "units_4", "mobile", "van", "med_yr_built", "min_pop"], inplace=True, axis=1)
tract_acs5_df["hmda_year"] = 2017 #set HMDA year (to match file spec sent from FFIEC Census flat file)
tract_acs5_df["fips"] = tract_acs5_df.apply(lambda x: x.state + x.county + x.tract ,axis=1)
tract_acs5_df.head() #check data

https://api.census.gov/data/2015/acs5?get=B19113_001E,B01001_001E,B25032_002E,B03002_003E,B25024_003E,B25024_002E,B25024_004E,B25024_005E,B25024_010E,B25024_011E,B25035_001E&for=tract:*&in=state:01
https://api.census.gov/data/2015/acs5?get=B19113_001E,B01001_001E,B25032_002E,B03002_003E,B25024_003E,B25024_002E,B25024_004E,B25024_005E,B25024_010E,B25024_011E,B25035_001E&for=tract:*&in=state:02
https://api.census.gov/data/2015/acs5?get=B19113_001E,B01001_001E,B25032_002E,B03002_003E,B25024_003E,B25024_002E,B25024_004E,B25024_005E,B25024_010E,B25024_011E,B25035_001E&for=tract:*&in=state:04
https://api.census.gov/data/2015/acs5?get=B19113_001E,B01001_001E,B25032_002E,B03002_003E,B25024_003E,B25024_002E,B25024_004E,B25024_005E,B25024_010E,B25024_011E,B25035_001E&for=tract:*&in=state:05
https://api.census.gov/data/2015/acs5?get=B19113_001E,B01001_001E,B25032_002E,B03002_003E,B25024_003E,B25024_002E,B25024_004E,B25024_005E,B25024_010E,B25024_011E,B25035_001E&for=tract:*&in=state:06
https://ap

https://api.census.gov/data/2015/acs5?get=B19113_001E,B01001_001E,B25032_002E,B03002_003E,B25024_003E,B25024_002E,B25024_004E,B25024_005E,B25024_010E,B25024_011E,B25035_001E&for=tract:*&in=state:47
https://api.census.gov/data/2015/acs5?get=B19113_001E,B01001_001E,B25032_002E,B03002_003E,B25024_003E,B25024_002E,B25024_004E,B25024_005E,B25024_010E,B25024_011E,B25035_001E&for=tract:*&in=state:48
https://api.census.gov/data/2015/acs5?get=B19113_001E,B01001_001E,B25032_002E,B03002_003E,B25024_003E,B25024_002E,B25024_004E,B25024_005E,B25024_010E,B25024_011E,B25035_001E&for=tract:*&in=state:49
https://api.census.gov/data/2015/acs5?get=B19113_001E,B01001_001E,B25032_002E,B03002_003E,B25024_003E,B25024_002E,B25024_004E,B25024_005E,B25024_010E,B25024_011E,B25035_001E&for=tract:*&in=state:50
https://api.census.gov/data/2015/acs5?get=B19113_001E,B01001_001E,B25032_002E,B03002_003E,B25024_003E,B25024_002E,B25024_004E,B25024_005E,B25024_010E,B25024_011E,B25035_001E&for=tract:*&in=state:51
https://ap

,tract_mfi,population,owner_occ,state,county,tract,total_1_to_4,med_age,min_pop_pct,hmda_year,fips
1,72727,1948,507,01,001,020100,724,36,12.577002,2017,01001020100
2,48750,2156,433,01,001,020200,785,39,59.554731,2017,01001020200
3,55766,2968,828,01,001,020300,1327,39,25.471698,2017,01001020300
4,69114,4423,1345,01,001,020400,1806,46,17.205517,2017,01001020400
5,75574,10763,2255,01,001,020500,3237,14,31.543250,2017,01001020500


In [64]:

tract_acs5_df.head()

,tract_mfi,population,owner_occ,state,county,tract,total_1_to_4,med_age,min_pop_pct,hmda_year,fips
1,72727,1948,507,01,001,020100,724,36,12.577002,2017,01001020100
2,48750,2156,433,01,001,020200,785,39,59.554731,2017,01001020200
3,55766,2968,828,01,001,020300,1327,39,25.471698,2017,01001020300
4,69114,4423,1345,01,001,020400,1806,46,17.205517,2017,01001020400
5,75574,10763,2255,01,001,020500,3237,14,31.543250,2017,01001020500


In [65]:
#need year, MSA fips, ffiec_mfi, tract_to_msa_pct, 
#need to get list of MSAs by state, wildcard does not work in call
#merge MSA from tract to cbsa
#get MSA MFI from ACS5 
msa_fields = ["B19113_001E"] #B19113_1 median family income (MFI)
msa_url = "https://api.census.gov/data/2015/acs5?get="
#https://api.census.gov/data/2015/acs5?get=B01001_001E&for=county:*&in=metropolitan%20statistical%20area/micropolitan%20statistical%20area:10100&in=state:46
first=True
for col in msa_fields:
    if first:
        msa_url = msa_url + col
        first = False
    else:
        msa_url = msa_url + "," + col
#msa_url = msa_url + "&for=county:*&in=metropolitan%20statistical%20area/micropolitan%20statistical%20area:10100&in=state:46"
msa_url ="https://api.census.gov/data/2015/acs5?get=B19113_001E&for=metropolitan%20statistical%20area/micropolitan%20statistical%20area:*"
print(msa_url)
acs5_msa = requests.get(msa_url)
acs5_msa_df = pd.DataFrame(json.loads(acs5_msa.content))
acs5_msa_df.drop([0], inplace=True)
acs5_msa_df.rename(columns={0:"msa_mfi", 1:"msa"},inplace=True)
acs5_msa_df.msa_mfi = acs5_msa_df.msa_mfi.map(lambda x: int(x))
acs5_msa_df.head()

https://api.census.gov/data/2015/acs5?get=B19113_001E&for=metropolitan%20statistical%20area/micropolitan%20statistical%20area:*


,msa_mfi,msa
1,12848,10260
2,18267,10380
3,20181,11640
4,20553,17620
5,18029,17640


In [76]:
#join MSA to tract to CBSA list
cbsa_mfi_df = cbsa_df.merge(acs5_msa_df, left_on="geoIdMsa", right_on="msa", how="left")
#join tract data to CBSA list
census_data = cbsa_mfi_df.merge(tract_acs5_df, on="fips", how="outer")
census_data["tract_to_msa_mfi_pct"] = census_data.apply(lambda x: (x.tract_mfi / x.msa_mfi)*100, axis=1)
census_data.drop(["state_y"], inplace=True, axis=1)
census_data.rename(columns={"state_x":"state"}, inplace=True)
#FIXME camel or underscore
cols = ["hmda_year", "msa", "metDivFp", "state", "county", "tract", "msa_mfi",
       "population", "min_pop_pct", "owner_occ", "total_1_to_4",
       "tract_mfi", "tract_to_msa_mfi_pct", "med_age"]
census_data.msa.fillna("", inplace=True)
census_data.tract_to_msa_mfi_pct.fillna(-1, inplace=True)
census_data.tract_to_msa_mfi_pct.fillna(-1, inplace=True)
census_data = census_data[cols]
census_data.hmda_year = 2017

census_data.tail()

,hmda_year,msa,metDivFp,state,county,tract,msa_mfi,population,min_pop_pct,owner_occ,total_1_to_4,tract_mfi,tract_to_msa_mfi_pct,med_age
75878,2017,31080,31084,06,037,113401,67989.0,5603.0,64.840264,1268.0,1568.0,77250.0,113.621321,57.0
75879,2017,41740,NaN,06,073,010103,75179.0,7008.0,87.785388,682.0,1352.0,50448.0,67.103845,43.0
75880,2017,12580,NaN,24,027,602202,87788.0,5366.0,28.494223,1525.0,1595.0,134083.0,152.734998,40.0
75881,2017,22180,NaN,37,051,003304,51775.0,6849.0,62.636881,1521.0,2245.0,50132.0,96.826654,38.0
75882,2017,,NaN,27,005,450300,NaN,5038.0,8.535133,1610.0,2331.0,64688.0,-1.000000,39.0


In [77]:
census_data.to_csv("output/cfpb_census_fields.txt", sep="|", index=False)

In [68]:
#check data integrity
#load HMDA LAR data from test file, this is for

In [70]:
census_data[(census_data.tract.isin(['020100','020300','020802']))&(census_data.county=='001')&(census_data.msa=='33860')]

,hmda_year,msa,metDivFp,state,county,tract,msa_mfi,population,min_pop_pct,owner_occ,total_1_to_4,tract_mfi,tract_to_msa_mfi_pct,med_age
20812,2017,33860,NaN,01,001,020300,59161.0,2968.0,25.471698,828.0,1327.0,55766.0,94.261422,39.0
49197,2017,33860,NaN,01,001,020100,59161.0,1948.0,12.577002,507.0,724.0,72727.0,122.930647,36.0
64795,2017,33860,NaN,01,001,020802,59161.0,10915.0,10.508475,3307.0,4303.0,73125.0,123.603387,18.0


In [71]:
#MSA income analysis
#move to new notebook

In [72]:
flat_file = pd.read_csv("output/ffiec_census_fields.txt", sep="|", dtype=object)
flat_file.head()
flat_file[(flat_file.tract.isin(['020100','020300','020802']))&(flat_file.county=='001')&(flat_file.msa=='33860')]

,hmda_year,msa,state,county,tract,ffiec_mfi,total_persons,min_pop_pct,owner_occupied,1_4_units,tract_mfi,tract_to_msa_mfi_pct,median_age
0,2017,33860,01,001,020100,59700,1948,12.58,507,724,72727,122.93,36
2,2017,33860,01,001,020300,59700,2968,25.47,828,1327,55766,94.26,39
8,2017,33860,01,001,020802,59700,10915,10.51,3307,4303,73125,123.60,18
